# Ejemplo descriptores globales para imágenes gris

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 14 de agosto de 2023

Se leen todas las imágenes jpg en la carpeta actual.
Se calculan todos los descriptres y se crea una matriz de descriptores.
Se calcula la distancia de todos contra todos y se imprimen los más cercanos.

## 1. Descriptor vector de intensidades

In [ ]:
import numpy
import cv2
import os

def agregar_texto(imagen, texto):
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.7
    fontThickness = 2
    position = (10, 30)
    fontColor = (50,50,50)
    cv2.putText(imagen, texto, position, fontFace, fontScale, fontColor, fontThickness, cv2.LINE_AA)

def mostrar_imagen(window_name, imagen, texto):
    MIN_WIDTH, MAX_WIDTH = 200, 800
    MIN_HEIGHT, MAX_HEIGHT = 200, 800
    if imagen.shape[0] > MAX_HEIGHT or imagen.shape[1] > MAX_WIDTH:
        # reducir tamaño
        fh = MAX_HEIGHT / imagen.shape[0]
        fw = MAX_WIDTH / imagen.shape[1]
        escala = min(fh, fw)
        imagen = cv2.resize(imagen, (0,0), fx=escala, fy=escala, interpolation=cv2.INTER_CUBIC)
    elif imagen.shape[0] < MIN_HEIGHT or imagen.shape[1] < MIN_WIDTH:
        # aumentar tamaño
        fh = MIN_HEIGHT / imagen.shape[0]
        fw = MIN_WIDTH / imagen.shape[1]
        escala = max(fh, fw)
        imagen = cv2.resize(imagen, (0,0), fx=escala, fy=escala, interpolation=cv2.INTER_NEAREST)
    # incluir el nombre sobre la imagen
    agregar_texto(imagen, texto)
    # mostrar en pantalla con el nomnbre
    cv2.imshow(window_name, imagen)

#funcion que retorna un vector
def vector_de_intensidades(archivo_imagen):
    imagen_1 = cv2.imread(archivo_imagen, cv2.IMREAD_GRAYSCALE)
    imagen_2 = cv2.resize(imagen_1, (20, 20), interpolation=cv2.INTER_AREA)
    # flatten convierte una matriz de nxm en un array de largo nxm
    descriptor_imagen = imagen_2.flatten()
    # mostrar una visualizacion del cálculo del descriptor
    global mostrar_imagenes
    if mostrar_imagenes:
        nombre = os.path.basename(archivo_imagen)
        mostrar_imagen("imagen_1", imagen_1, nombre)
        mostrar_imagen("imagen_2", imagen_2, nombre)
        cv2.waitKey()
        cv2.destroyAllWindows()
    return descriptor_imagen

def calcular_descriptores(metodo_descriptor, imagenes_dir):
    lista_nombres = []
    matriz_descriptores = []    
    for nombre in os.listdir(imagenes_dir):
        archivo_imagen = "{}/{}".format(imagenes_dir, nombre)
        descriptor_imagen = metodo_descriptor(archivo_imagen)
        # agregar descriptor a la matriz de descriptores
        if len(matriz_descriptores) == 0:
            matriz_descriptores = descriptor_imagen
        else:
            matriz_descriptores = numpy.vstack([matriz_descriptores, descriptor_imagen])
        # agregar nombre del archivo a la lista de nombres
        lista_nombres.append(nombre)
    return lista_nombres, matriz_descriptores

def imprimir_matriz_descriptores(nombres, descriptores):
    print("matriz de descriptores (filas={}, columnas={})".format(descriptores.shape[0], descriptores.shape[1]))
    numpy.set_printoptions(precision=3, floatmode="fixed", suppress=True, threshold=15, edgeitems=5, linewidth=100)
    print("filas de la matriz:")
    for i in range(len(nombres)):
        print("  {}) {:>15s} {}".format(i, nombres[i], descriptores[i]))
    print()

        
imagenes_dir = "imagenes"
mostrar_imagenes = True
nombres, descriptores = calcular_descriptores(vector_de_intensidades, imagenes_dir)

imprimir_matriz_descriptores(nombres, descriptores)

### Comparar descriptores

Compara todos los vectores entre sí con la función `cdist`.  
Requiere instalar scipy (`conda install scipy`).  
Se pueden cambiar la función de distancia a usar con el parámetro `metric="nombre_distancia"`.  
Ver https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html

In [ ]:
import scipy.spatial

matriz_distancias = scipy.spatial.distance.cdist(descriptores, descriptores, metric='cityblock')

print("matriz de distancias ({}x{},todos contra todos)=\n{}".format(matriz_distancias.shape[0],matriz_distancias.shape[1],matriz_distancias))

### Imprimir los más cercanos

Para cada imagen se muestra la imagen más cercana (que no sea sí misma).

In [ ]:
import pandas

def imprimir_cercanos(lista_nombres, matriz_distancias):
    # completar la diagonal con un valor muy grande para que el mas cercano no sea si mismo
    numpy.fill_diagonal(matriz_distancias, numpy.inf)

    # obtener la posicion del mas cercano por fila
    posiciones_minimas = numpy.argmin(matriz_distancias, axis=1)
    valores_minimos = numpy.amin(matriz_distancias, axis=1)

    resultado_mas_cercanos = []

    for i in range(len(matriz_distancias)):
        query = lista_nombres[i]
        distancia = valores_minimos[i]
        mas_cercano = lista_nombres[posiciones_minimas[i]]
        resultado_mas_cercanos.append([query, mas_cercano, distancia])

    df = pandas.DataFrame(resultado_mas_cercanos, columns=["query", "más_cercana", "distancia"])
    print(df.to_string(index=False,justify='center'))

imprimir_cercanos(nombres, matriz_distancias)

## 2. Descriptor con ecualización del histograma y  distancia euclidiana

Explicación de la ecualización del histograma: https://docs.opencv.org/3.4/d4/d1b/tutorial_histogram_equalization.html

In [ ]:
def vector_de_intensidades_equalize(archivo_imagen):
    imagen_1 = cv2.imread(archivo_imagen, cv2.IMREAD_GRAYSCALE)
    imagen_2 = cv2.equalizeHist(imagen_1)
    imagen_2 = cv2.resize(imagen_2, (4, 4), interpolation=cv2.INTER_AREA)
    descriptor_imagen = imagen_2.flatten()
    global mostrar_imagenes
    if mostrar_imagenes:
        mostrar_imagen("imagen_1", imagen_1, archivo_imagen)
        mostrar_imagen("imagen_2", imagen_2, "")
        cv2.waitKey()
        cv2.destroyAllWindows()
    return descriptor_imagen

nombres2, descriptores2 = calcular_descriptores(vector_de_intensidades_equalize, imagenes_dir)

matriz_distancias2 = scipy.spatial.distance.cdist(descriptores2, descriptores2, metric='euclidean')

imprimir_cercanos(nombres2, matriz_distancias2)

## 3. Descriptor OMD y distancia de Hamming

Reemplazar el valor gris de la zona por su posicion en un arreglo ordenado.

In [ ]:
def vector_de_intensidades_omd(archivo_imagen):
    imagen_1 = cv2.imread(archivo_imagen, cv2.IMREAD_GRAYSCALE)
    imagen_2 = cv2.resize(imagen_1, (4, 4), interpolation=cv2.INTER_AREA)
    descriptor_imagen = imagen_2.flatten()
    posiciones = numpy.argsort(descriptor_imagen)
    for i in range(len(posiciones)):
        descriptor_imagen[posiciones[i]] = i
    return descriptor_imagen

nombres_omd, descriptores_omd = calcular_descriptores(vector_de_intensidades_omd, imagenes_dir)

imprimir_matriz_descriptores(nombres_omd, descriptores_omd)

matriz_distancias_omd = scipy.spatial.distance.cdist(descriptores_omd, descriptores_omd, metric='hamming')

imprimir_cercanos(nombres_omd, matriz_distancias_omd)

## 4. Descriptor Histogramas por zona

In [ ]:
def dibujar_histograma(img, histograma, limites):
    cv2.rectangle(img, (0, 0), (img.shape[1]-1, img.shape[0]-1), (255,200,120), 1)
    pos_y_base = img.shape[0] - 6
    max_altura = img.shape[0] - 10
    nbins = len(histograma)
    for i in range(nbins):
        desde_x = int(img.shape[1] / nbins * i)
        hasta_x = int(img.shape[1] / nbins * (i+1))
        altura = int(histograma[i] * max_altura)
        g = int((limites[i]+limites[i+1])/2)
        color = (g, g, g)
        pt1 = (desde_x, pos_y_base + 5)
        pt2 = (hasta_x-1, pos_y_base - altura)
        cv2.rectangle(img, pt1, pt2, color, -1)
    cv2.line(img, (0, pos_y_base), (img.shape[1] - 1, pos_y_base), (120,120,255), 1)

def histograma_por_zona(archivo_imagen):
    # divisiones
    num_zonas_x = 2
    num_zonas_y = 2 
    num_bins_por_zona = 8
    ecualizar = True
    # leer imagen
    imagen = cv2.imread(archivo_imagen, cv2.IMREAD_GRAYSCALE)
    if ecualizar:
        imagen = cv2.equalizeHist(imagen)
    # para dibujar los histogramas
    global mostrar_imagenes
    imagen_hists = numpy.full((imagen.shape[0], imagen.shape[1], 3), (200,255,200), dtype=numpy.uint8)
    # procesar cada zona
    descriptor = []
    for j in range(num_zonas_y):
        desde_y = int(imagen.shape[0] / num_zonas_y * j)
        hasta_y = int(imagen.shape[0] / num_zonas_y * (j+1))
        for i in range(num_zonas_x):
            desde_x = int(imagen.shape[1] / num_zonas_x * i)
            hasta_x = int(imagen.shape[1] / num_zonas_x * (i+1))
            # recortar zona de la imagen
            zona = imagen[desde_y : hasta_y, desde_x : hasta_x]
            # histograma de los pixeles de la zona
            histograma, limites = numpy.histogram(zona, bins=num_bins_por_zona, range=(0,255))
            # normalizar histograma (bins suman 1)
            histograma = histograma / numpy.sum(histograma)
            # agregar descriptor de la zona al descriptor global
            descriptor.extend(histograma)
            # dibujar histograma de la zona
            if mostrar_imagenes:
                zona_hist = imagen_hists[desde_y : hasta_y, desde_x : hasta_x]
                dibujar_histograma(zona_hist, histograma, limites)
    # mostrar imagen con histogramas
    if mostrar_imagenes:
        nombre = os.path.basename(archivo_imagen)
        mostrar_imagen("imagen", imagen, nombre)
        mostrar_imagen("histograma", imagen_hists, "")
        cv2.waitKey()
        cv2.destroyAllWindows()
    return descriptor

nombres_hist, descriptores_hist = calcular_descriptores(histograma_por_zona, imagenes_dir)

imprimir_matriz_descriptores(nombres_hist, descriptores_hist)

matriz_distancias_hist = scipy.spatial.distance.cdist(descriptores_hist, descriptores_hist, metric='cityblock')

imprimir_cercanos(nombres_hist, matriz_distancias_hist)

## 5. Descriptor HOG

Se aplica un filtro de sobel y sobre los pixeles que son de borde se calcula el ángulo del borde.

Usualmente es buena idea reducir un poco la imagen o aplicar un filtro gaussiano para quitar un poco de ruido y detalles de la imagen.

**Implementación muy lenta!** Recorre cada pixel de la imagen con python. Para mejorar la velocidad se deben usar funciones de numpy (para recorrer la imagen con C++).

In [ ]:
import math

def angulos_en_zona(imgBordes, imgSobelX, imgSobelY):
    # calcular angulos de la zona
    # recorre pixel por pixel (muy lento!)
    angulos = []
    for row in range(imgBordes.shape[0]):
        for col in range(imgBordes.shape[1]):
            # si es un pixel de borde (magnitud del gradiente > umbral)
            if imgBordes[row][col] > 0:
                dx = imgSobelX[row][col]
                dy = imgSobelY[row][col]
                angulo = 90
                if dx != 0:
                    # un numero entre -180 y 180
                    angulo = math.degrees(numpy.arctan(dy/dx))
                    # dejar en el rango -90 a 90
                    if angulo <= -90:
                        angulo += 180
                    if angulo > 90:
                        angulo -= 180
                angulos.append(angulo)
    return angulos

def angulos_por_zona(archivo_imagen):
    # divisiones
    num_zonas_x = 8
    num_zonas_y = 8 
    num_bins_por_zona = 9
    threshold_magnitud_gradiente = 150
    # leer imagen
    imagen = cv2.imread(archivo_imagen, cv2.IMREAD_GRAYSCALE)
    # calcular filtro de sobel (usar cv2.GaussianBlur para borrar ruido)
    imagen = cv2.equalizeHist(imagen)
    imagen = cv2.GaussianBlur(imagen, (5,5), 0, 0)
    sobelX = cv2.Sobel(imagen, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=3)
    sobelY = cv2.Sobel(imagen, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=3)
    magnitud = numpy.sqrt(numpy.square(sobelX) + numpy.square(sobelY))
    th, bordes = cv2.threshold(magnitud, threshold_magnitud_gradiente, 255, cv2.THRESH_BINARY)
    # para ver los histogramas
    global mostrar_imagenes
    imagen_hists = numpy.full((imagen.shape[0], imagen.shape[1], 3), (200,210,255), dtype=numpy.uint8)
    # procesar cada zona
    descriptor = []
    for j in range(num_zonas_y):
        desde_y = int(imagen.shape[0] / num_zonas_y * j)
        hasta_y = int(imagen.shape[0] / num_zonas_y * (j+1))
        for i in range(num_zonas_x):
            desde_x = int(imagen.shape[1] / num_zonas_x * i)
            hasta_x = int(imagen.shape[1] / num_zonas_x * (i+1))
            # calcular angulos de la zona
            angulos = angulos_en_zona(bordes[desde_y : hasta_y, desde_x : hasta_x],
                                      sobelX[desde_y : hasta_y, desde_x : hasta_x],
                                      sobelY[desde_y : hasta_y, desde_x : hasta_x])
            # histograma de los angulos de la zona
            histograma, limites = numpy.histogram(angulos, bins=num_bins_por_zona, range=(-90,90))
            # normalizar histograma (bins suman 1)
            if numpy.sum(histograma) != 0:
                histograma = histograma / numpy.sum(histograma)
            # agregar descriptor de la zona al descriptor global
            descriptor.extend(histograma)
            # dibujar histograma de la zona
            if mostrar_imagenes:
                zona_hist = imagen_hists[desde_y : hasta_y, desde_x : hasta_x]
                limites = (limites + 180) / 360 * 255
                dibujar_histograma(zona_hist, histograma, limites)
    # mostrar imagen con histogramas
    if mostrar_imagenes:
        nombre = os.path.basename(archivo_imagen)
        mostrar_imagen("imagen", imagen, nombre)
        mostrar_imagen("bordes", bordes, nombre)
        mostrar_imagen("histograma", imagen_hists, "")
        cv2.waitKey()
        cv2.destroyAllWindows()
    return descriptor

nombres_sobel, descriptores_sobel = calcular_descriptores(angulos_por_zona, imagenes_dir)

imprimir_matriz_descriptores(nombres_sobel, descriptores_sobel)

matriz_distancias_sobel = scipy.spatial.distance.cdist(descriptores_sobel, descriptores_sobel, metric='cityblock')

imprimir_cercanos(nombres_sobel, matriz_distancias_sobel)